<h2 align=center> Facial Expression Recognition</h2>

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical

from IPython.display import SVG, Image
from livelossplot.tf_keras import PlotLossesCallback
import tensorflow as tf

import cv2
from skimage.transform import resize

print("Tensorflow version:", tf.__version__)

## Getting total number of images of each category

In [ ]:
import os

for expression in os.listdir("../fer2013-kaggle/train"):
    print(str(len(os.listdir("../fer2013-kaggle/train/" + expression))) + " " + expression + " images")

## Generate Training and Validation Batches

In [ ]:
from tensorflow.python.lib.io import file_io

def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
    # dataset: Data path
def get_data(dataset):
    data = pd.read_csv(dataset)
    pixels = data['Paths'].tolist()
    images = np.empty((len(data), img_size, img_size, 3))
    i = 0

    for pixel_path in pixels:
        if i%5000 ==0:
            print(i," done")
        single_image = cv2.imread('../'+pixel_path, cv2.IMREAD_GRAYSCALE)
        #single_image = np.asarray(single_image).reshape(48, 48) # Dimension: 48x48
        single_image = resize(single_image, (img_size, img_size), order = 3, mode = 'constant') # Dimension: 139x139x3 (Bicubic)
        ret = np.empty((img_size, img_size, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = to_categorical(data['Expression'])

    return images, labels    

In [ ]:
img_size = 224

In [ ]:
# Data preparation
train_data_x, train_data_y  = get_data('../fer2013-kaggle/Train.csv')
val_data  = get_data('../fer2013-kaggle/Test.csv')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,                                        
        fill_mode='nearest',
    )


datagen_validation = ImageDataGenerator(
    rescale = 1 / 255.,
)

train_generator = train_preprocessor.flow(
    train_data_x,
    train_data_y,
    batch_size  = batch_size)


## Create CNN Model

In [ ]:
resnet = tf.keras.applications.ResNet50V2(weights='imagenet', 
                                        input_shape=(img_size, img_size, 3),
                                        include_top= False)

In [ ]:
resnet.summary()

In [ ]:
resnet.trainable = True

for layer in resnet.layers[:-20]:
    layer.trainable = False

In [ ]:
model = Sequential([
                      resnet,
                      Dropout(.25),
                      BatchNormalization(),
                      Flatten(),
                      Dense(64, activation='relu'),
                      BatchNormalization(),
                      Dropout(.5),
                      Dense(7,activation='softmax')
                    ])

In [ ]:
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training and Evaluating Model

In [ ]:
epochs = 20

checkpoint = ModelCheckpoint(
                            "../Model weights/Resnet_model.h5", monitor='val_accuracy',
                             save_best_only=True, mode='max', verbose=1
                            )

# Create Early Stopping Callback to monitor the accuracy
Early_Stopping = EarlyStopping(monitor = 'val_accuracy', patience = 5, restore_best_weights = True)

# Create ReduceLROnPlateau Callback to reduce overfitting by decreasing learning
Reducing_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')

callbacks = [Early_Stopping, Reducing_LR, checkpoint]

steps_per_epoch = train_generator.n//train_generator.batch_size


In [ ]:
history = model.fit(
       x = train_data_x,
    y=train_data_y,
    steps_per_epoch=len(train_data_x) // batch_size,
    epochs=epochs,
    validation_data = val_data,
    callbacks= callbacks
)

In [ ]:
plot_curves(history)

## Class for loading model and weights

In [ ]:
from tensorflow.keras.models import model_from_json
import numpy as np

import tensorflow as tf


class FacialExpressionModel(object):

    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]

    def __init__(self, model_file):
        # load model from JSON file
        self.loaded_model = load_model(model_file)

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]
    
    def predict(self, img):
        self.preds = self.loaded_model.predict(img)
        return self.preds


## Getting frames and doing prediction

In [ ]:
  
import cv2
import numpy as np

facec = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')
model_facial_Exp = FacialExpressionModel("../Model weights/RESNET_model.h5")
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            single_image = resize(single_image, (img_size, img_size), order = 3, mode = 'constant') # Dimension: 139x139x3 (Bicubic)
            ret = np.empty((img_size, img_size, 3))  
            ret[:, :, 0] = single_image
            ret[:, :, 1] = single_image
            ret[:, :, 2] = single_image
            pred = model_facial_Exp.predict_emotion(ret[np.newaxis, :, :])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr

## Function for showing output video

In [ ]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [ ]:
import seaborn as sns 
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:


# Get the true labels and predicted labels for the validation set
validation_labels = np.argmax(val_data[1], axis=1)
validation_pred_probs = model.predict(val_data[0])
validation_pred_labels = np.argmax(validation_pred_probs, axis=1)

# Compute the confusion matrix
confusion_mtx = confusion_matrix(validation_labels, validation_pred_labels)
class_names = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]#list(train_generator.class_indices.keys())
sns.set()
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(validation_labels, validation_pred_labels))